In [46]:
import datacube
from datacube.utils import geometry
import odc.ui
from odc.ui import to_png_data
from odc.ui import mk_data_uri
from odc.ui import to_jpeg_data
from odc.algo import to_rgba, is_rgb
import xarray as xr

#dea tools
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.dask import create_local_dask_cluster

#geodata plotting
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

#map
import folium
import leafmap.foliumap as leafmapfol
import leafmap
from ipyleaflet import Map, basemaps
from ipywidgets import widgets as w
import ipyleaflet

#typing
from typing import Tuple
from typing import Union
from typing import Optional

#streamlit
#from streamlit_folium import folium_static
#import streamlit as st
#from streamlit_option_menu import option_menu
from datacube.utils import geometry
from deafrica_tools.dask import create_local_dask_cluster
#from st_btn_select import st_btn_select

#text
#from annotated_text import annotated_text

#time
import datetime
import time

#image
from PIL import Image
import numpy as np

#modeling
import itertools
import joblib
from deafrica_tools.classification import predict_xr
from deafrica_tools.spatial import xr_rasterize

from predictions_tomap import *
import sys
sys.path.append('./Numeric data models/')
#print(sys.path)
from feature_layers import *
import json

In [47]:
dc = datacube.Datacube(app='predictions_tomap')
crops_code = json.load(open('crops_code.json'))

In [48]:
year = '2020'
input_polygon = gpd.read_file('tadla_test.geojson')
input_polygon['id'] = input_polygon.index
geom = geometry.Geometry(input_polygon.iloc[0].geometry.__geo_interface__,geometry.CRS(f'EPSG:{input_polygon.crs.to_epsg()}'))
lon1, lat1, lon2, lat2 = input_polygon.total_bounds
lon = (lon1 + lon2) / 2
lat = (lat1 + lat2) / 2

In [49]:
model_gm_1 = joblib.load('/home/jovyan/cropmapping_open_datacube_dea/Models/rf_gm_1.joblib')
features_gm_1 =  ['blue_S1_n',
                 'green_S1_n',
                 'red_S1_n',
                 'nir_S1_n',
                 'swir_1_S1_n',
                 'swir_2_S1_n',
                 'red_edge_1_S1_n',
                 'red_edge_2_S1_n',
                 'red_edge_3_S1_n',
                 'sdev_S1_n',
                 'bcdev_S1_n',
                 'edev_S1_n',
                 'NDVI_S1_n',
                 'SAVI_S1_n',
                 'NDMI_S1_n',
                 'blue_S2_n',
                 'green_S2_n',
                 'red_S2_n',
                 'nir_S2_n',
                 'swir_1_S2_n',
                 'swir_2_S2_n',
                 'red_edge_1_S2_n',
                 'sdev_S2_n',
                 'bcdev_S2_n',
                 'edev_S2_n',
                 'NDVI_S2_n',
                 'NDMI_S2_n']

In [50]:
queries = json.load(open('/home/jovyan/cropmapping_open_datacube_dea/Numeric data models/models_prediction_data_queries.json'))
queries['query_gm_1'].update({'geopolygon': geom})
queries['query_gm_1'].update({'time': year})

In [51]:
predict_input = gm_1(queries['query_gm_1'])
predict_input = predict_input[features_gm_1]

In [52]:
predicted = predict_xr(model_gm_1,
                       predict_input,
                       #proba=True,
                       clean=True,
                       #persist=True,
                       return_input=True)

predicting...
   input features...


In [53]:
#agri_geom_masking
agri_mask = dc.load(product='crop_mask_northern', time=('2019'), like=predict_input.geobox).filtered.squeeze()
predicted = predicted.where(agri_mask==1)
geom_mask = xr_rasterize(input_polygon.iloc[[0]], predicted)
predicted = predicted.where(geom_mask)

In [54]:
im_layer_0 = mk_image_overlay(predicted.where(predicted.Predictions==0), clamp=900, bands=['red_S1_n', 'blue_S1_n', 'green_S1_n'] , fmt='png')
im_layer_1 = mk_image_overlay(predicted.where(predicted.Predictions==1), clamp=900, bands=['red_S1_n', 'blue_S1_n', 'green_S1_n'] , fmt='png')
im_layer_5 = mk_image_overlay(predicted.where(predicted.Predictions==5), clamp=900, bands=['red_S1_n', 'blue_S1_n', 'green_S1_n'] , fmt='png')

In [55]:
for y, x in itertools.product(range(im_layer_0[1].values.shape[0]), range(im_layer_0[1].values.shape[1])):
    if im_layer_0[1].values[y,x,3:][0] == 255:
        im_layer_0[1].values[y,x] = np.array(crops_code['durum_wheat'])
    else:
        im_layer_0[1].values[y,x] = np.array([0,0,0,0])

        
for y, x in itertools.product(range(im_layer_1[1].values.shape[0]), range(im_layer_1[1].values.shape[1])):
    if im_layer_1[1].values[y,x,3:][0] == 255:
        im_layer_1[1].values[y,x] = np.array(crops_code['soft_wheat'])
    else:
        im_layer_1[1].values[y,x] = np.array([0,0,0,0])

        
for y, x in itertools.product(range(im_layer_5[1].values.shape[0]), range(im_layer_5[1].values.shape[1])):
    if im_layer_5[1].values[y,x,3:][0] == 255:
        im_layer_5[1].values[y,x] = np.array(crops_code['barley'])
    else:
        im_layer_5[1].values[y,x] = np.array([0,0,0,0])

In [56]:
im_0 = Image.fromarray(im_layer_0[1].values)
im_1 = Image.fromarray(im_layer_1[1].values)
im_5 = Image.fromarray(im_layer_5[1].values)
im_0.save("im_0.png")
im_1.save("im_1.png")
im_5.save("im_5.png")

In [57]:
m = leafmap.Map(basemap=basemaps.Esri.WorldImagery, center=(lon,lat), zoom=12, layout=w.Layout(width='1000px', height='500px'), layer_control=True)
image_0 = leafmap.ImageOverlay(url="im_0.png", bounds=xr_bounds(im_layer_0[1]), name='durum_wheat')
image_1 = leafmap.ImageOverlay(url="im_1.png", bounds=xr_bounds(im_layer_1[1]), name='soft_wheat')
image_5 = leafmap.ImageOverlay(url="im_5.png", bounds=xr_bounds(im_layer_5[1]), name='barley')
m.add_gdf(input_polygon, layer_name="AOI")
m.add_layer(image_0)
m.add_layer(image_1)
m.add_layer(image_5)
m

Map(center=[-6.907138824462891, 32.267752027456744], controls=(ZoomControl(options=['position', 'zoom_in_text'…